In [8]:
!pip3 install tqdm
!pip3 install twilio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [4]:
query = 'Montevideo'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=427cd0f9bf234a1a8fd122353242104&q=Montevideo&days=1&aqi=no&alerts=no'

In [4]:
response = requests.get(url_clima).json()

In [5]:
response

{'location': {'name': 'Montevideo',
  'region': 'Montevideo',
  'country': 'Uruguay',
  'lat': -34.86,
  'lon': -56.17,
  'tz_id': 'America/Montevideo',
  'localtime_epoch': 1713711132,
  'localtime': '2024-04-21 11:52'},
 'current': {'last_updated_epoch': 1713710700,
  'last_updated': '2024-04-21 11:45',
  'temp_c': 20.0,
  'temp_f': 68.0,
  'is_day': 1,
  'condition': {'text': 'Overcast',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png',
   'code': 1009},
  'wind_mph': 11.9,
  'wind_kph': 19.1,
  'wind_degree': 160,
  'wind_dir': 'SSE',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.9,
  'precip_in': 0.04,
  'humidity': 100,
  'cloud': 100,
  'feelslike_c': 20.0,
  'feelslike_f': 68.0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 5.0,
  'gust_mph': 16.3,
  'gust_kph': 26.3},
 'forecast': {'forecastday': [{'date': '2024-04-21',
    'date_epoch': 1713657600,
    'day': {'maxtemp_c': 19.6,
     'maxtemp_f': 67.2,
     'mintemp_c': 17.4,
     'mintemp_f': 6

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [7]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [8]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [9]:
response['forecast']['forecastday'][0]['hour'][2]['time'].split()[0]

'2024-04-21'

In [10]:
int(response['forecast']['forecastday'][0]['hour'][2]['time'].split()[1].split(':')[0]) #hora

2

In [11]:
response['forecast']['forecastday'][0]['hour'][2]['condition']['text']

'Patchy rain nearby'

In [12]:
response['forecast']['forecastday'][0]['hour'][2]['temp_c']

18.4

In [13]:
response['forecast']['forecastday'][0]['hour'][2]['will_it_rain']

1

In [14]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

76

In [15]:
def get_forecast(response, i):
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]) #hora
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return fecha, hora, condicion, tempe, rain, prob_rain

In [16]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'):
    datos.append(get_forecast(response, i))

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 72838.85it/s]


In [20]:
datos

[('2024-04-21', 0, 'Patchy rain nearby', 18.2, 1, 82),
 ('2024-04-21', 1, 'Cloudy ', 18.1, 0, 0),
 ('2024-04-21', 2, 'Patchy rain nearby', 18.4, 1, 76),
 ('2024-04-21', 3, 'Overcast ', 18.5, 0, 0),
 ('2024-04-21', 4, 'Overcast ', 18.6, 0, 0),
 ('2024-04-21', 5, 'Patchy rain nearby', 18.4, 1, 84),
 ('2024-04-21', 6, 'Light rain shower', 18.3, 1, 100),
 ('2024-04-21', 7, 'Patchy rain nearby', 18.3, 1, 100),
 ('2024-04-21', 8, 'Patchy rain nearby', 18.4, 1, 81),
 ('2024-04-21', 9, 'Cloudy ', 19.0, 0, 0),
 ('2024-04-21', 10, 'Partly Cloudy ', 18.8, 0, 0),
 ('2024-04-21', 11, 'Overcast', 20.0, 1, 100),
 ('2024-04-21', 12, 'Patchy rain nearby', 18.6, 1, 100),
 ('2024-04-21', 13, 'Patchy rain nearby', 19.6, 1, 100),
 ('2024-04-21', 14, 'Patchy rain nearby', 19.5, 1, 100),
 ('2024-04-21', 15, 'Patchy rain nearby', 19.3, 1, 72),
 ('2024-04-21', 16, 'Patchy rain nearby', 18.4, 1, 71),
 ('2024-04-21', 17, 'Patchy rain nearby', 18.1, 1, 82),
 ('2024-04-21', 18, 'Patchy rain nearby', 17.7, 0, 62),


In [21]:
datos[0]

('2024-04-21', 0, 'Patchy rain nearby', 18.2, 1, 82)

In [22]:
col = ['Fecha', 'Hora', 'Condicion', 'Tempe', 'Lluvia', 'Prob_lluvia']

In [23]:
df = pd.DataFrame(datos, columns=col)
df

,Fecha,Hora,Condicion,Tempe,Lluvia,Prob_lluvia
0,2024-04-21,0,Patchy rain nearby,18.2,1,82
1,2024-04-21,1,Cloudy,18.1,0,0
2,2024-04-21,2,Patchy rain nearby,18.4,1,76
3,2024-04-21,3,Overcast,18.5,0,0
4,2024-04-21,4,Overcast,18.6,0,0
5,2024-04-21,5,Patchy rain nearby,18.4,1,84
6,2024-04-21,6,Light rain shower,18.3,1,100
7,2024-04-21,7,Patchy rain nearby,18.3,1,100
8,2024-04-21,8,Patchy rain nearby,18.4,1,81
9,2024-04-21,9,Cloudy,19.0,0,0


In [24]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']<22)]
df_rain = df_rain[['Hora', 'Condicion']]
df_rain.set_index('Hora', inplace=True)
df_rain

,Condicion
Hora,
7,Patchy rain nearby
8,Patchy rain nearby
11,Overcast
12,Patchy rain nearby
13,Patchy rain nearby
14,Patchy rain nearby
15,Patchy rain nearby
16,Patchy rain nearby
17,Patchy rain nearby


In [25]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2024-04-21 en Montevideo es : \n\n\n                Condicion\nHora                    \n7     Patchy rain nearby\n8     Patchy rain nearby\n11              Overcast\n12    Patchy rain nearby\n13    Patchy rain nearby\n14    Patchy rain nearby\n15    Patchy rain nearby\n16    Patchy rain nearby\n17    Patchy rain nearby\n19    Patchy rain nearby\n20    Patchy rain nearby\n21    Patchy rain nearby'

In [28]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+59895040138'
                 )

print('Mensaje Enviado ' + message.sid)

TwilioRestException: HTTP 400 error: Unable to create record: The number +5989535XXXX is unverified. Trial accounts cannot send messages to unverified numbers; verify +5989535XXXX at twilio.com/user/account/phone-numbers/verified, or purchase a Twilio number to send messages to unverified numbers